# 1. 환경설정 / 입력 데이터 로드

## 1.1 라이브러리 및 공통 설정

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# 재현성
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# pandas 출력 옵션(필요 시 조절)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)
pd.set_option("display.max_rows", 120)

print("✅ libs/settings ready")

✅ libs/settings ready


## 1.2 Feature 파일 로드

In [2]:
FEATURE_DIR = Path("../artifacts/features")
FEATURE_PATH = FEATURE_DIR / "daily_feature_v3.csv"

assert FEATURE_PATH.exists(), f"daily_feature_v3.csv not found: {FEATURE_PATH}"

df = pd.read_csv(FEATURE_PATH)
print("Loaded shape:", df.shape)

# date normalize
df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.normalize()

# drop invalid keys
df = df.dropna(subset=["uuid", "date"]).copy()

# sort (user timeline 보장)
df = df.sort_values(["uuid", "date"]).reset_index(drop=True)

# 기본 파생 (v1 잔재 제거: is_weekend은 선택사항이지만 있으면 유용)
df["is_weekend"] = df["date"].dt.weekday >= 5

# cold start stage indexing
first_day = df.groupby("uuid")["date"].transform("min")
df["day_idx"] = (df["date"] - first_day).dt.days + 1

df["cold_stage"] = np.select(
    [
        df["day_idx"].between(1, 4),
        df["day_idx"].between(5, 8), 
        df["day_idx"].between(9, 15),
        df["day_idx"] >= 16,
    ],
    ["ONBOARD", "WARMUP", "SEMI_READY", "READY"],
    default="ONBOARD"
)

print("Users:", df["uuid"].nunique())
print("User-Days:", df[["uuid","date"]].drop_duplicates().shape[0])
df.head(3)

Loaded shape: (9057, 54)
Users: 342
User-Days: 9057


,uuid,date,Screen,UserAct,Notif,unlock_cnt,daily_event_cnt,has_activity,qc_core_low_cov,qc_core_very_low_activity,qc_rhythm_low_cov,qc_gap_low_cov,qc_meta_low_heartbeat,qc_meta_retry_warn,qc_meta_queue_warn,partial_signal_raw,travel_signal_raw,tz_change_signal,rhythm_event_cnt,rhythm_low_coverage,night_ratio,hour_entropy,day_ratio,evening_ratio,peak_hour,peak_ratio,gap_event_cnt,gap_low_coverage,gap_max,gap_p95,gap_cnt_2h,gap_cnt_6h,gap_long_ratio,overnight_gap,first_hour,last_hour,heartbeat_cnt,retry_max,queue_max,tz_offset_min,tz_offset_max,tz_changed,qc_last_ts_max,daily_event_cnt_d1,night_ratio_d1,hour_entropy_d1,gap_max_d1,gap_p95_d1,gap_cnt_2h_d1,gap_cnt_6h_d1,travel_signal_raw_d1,partial_signal_raw_d1,tz_change_signal_d1,daily_event_cnt_r1,is_weekend,day_idx,cold_stage
0,u072142,2016-06-19,68,94,267,0,162,True,False,False,False,False,NaN,NaN,NaN,0.0,0.0,0.0,162.0,False,0.005376,1.939641,0.358025,0.641975,18.0,0.265432,162.0,False,2.120157,0.195141,1.0,0.0,0.088340,NaN,15.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1,ONBOARD
1,u072142,2016-06-20,48,208,92,0,256,True,False,False,False,False,NaN,NaN,NaN,0.0,0.0,0.0,256.0,False,0.167857,2.608977,0.492188,0.328125,15.0,0.156250,256.0,False,6.964134,0.289940,1.0,1.0,0.290172,0.147440,0.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.0,0.162481,0.669335,4.843977,0.094799,0.0,1.0,0.0,0.0,0.0,0.580247,False,2,ONBOARD
2,u072142,2016-06-21,90,84,132,0,174,True,False,False,False,False,NaN,NaN,NaN,0.0,0.0,0.0,174.0,False,0.146465,2.422839,0.396552,0.442529,21.0,0.212644,174.0,False,6.299334,0.547004,3.0,1.0,0.469780,0.007304,0.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-82.0,-0.021392,-0.186138,-0.664800,0.257064,2.0,0.0,0.0,0.0,0.0,-0.320312,False,3,ONBOARD


## 1.3 입력 스키마 검증 및 정합성 체크

In [3]:
# ✅ v3에서 "최소 모델 구동"에 필요한 컬럼들
REQUIRED_COLS_V3 = [
    "uuid", "date",
    "daily_event_cnt", "has_activity",

    # v3 QC (v1의 low_coverage_day를 쓰지 않음)
    "qc_core_low_cov", "qc_rhythm_low_cov", "qc_gap_low_cov",

    # v3 핵심 context signals
    "partial_signal_raw", "travel_signal_raw", "tz_change_signal",
]

missing = [c for c in REQUIRED_COLS_V3 if c not in df.columns]
if missing:
    raise ValueError(f"[Schema] Missing v3 required columns: {missing}")

# ✅ v1_FE 잔재 방지(있어도 쓰지 말자)
V1_LEGACY_COLS = ["low_coverage_day"]
alias_found = [c for c in ["rhythm_low_coverage","gap_low_coverage"] if c in df.columns]
if alias_found:
    print("ℹ️ legacy-like QC cols exist (ignored; using qc_*):", alias_found)

print("✅ Required columns OK (v3)")

ℹ️ legacy-like QC cols exist (ignored; using qc_*): ['rhythm_low_coverage', 'gap_low_coverage']
✅ Required columns OK (v3)


## 1.4 타입 표준화

In [4]:
# bool류: pandas BooleanDtype로 통일(NA 허용)
BOOL_COLS = [
    "has_activity", "is_weekend",
    "qc_core_low_cov", "qc_rhythm_low_cov", "qc_gap_low_cov",
    "qc_meta_low_heartbeat", "qc_meta_retry_warn", "qc_meta_queue_warn",  # 있으면 자동 처리
]

for c in BOOL_COLS:
    if c in df.columns:
        df[c] = df[c].astype("boolean")

# 숫자형: to_numeric로 통일
NUM_COLS = [
    "daily_event_cnt",
    "unlock_cnt",            # ✅ 추가: 기기 사용의 핵심
    "partial_signal_raw", "travel_signal_raw", "tz_change_signal",
    "night_ratio", "hour_entropy",
    "gap_max", "gap_p95", "gap_cnt_2h", "gap_cnt_6h", "gap_long_ratio", "overnight_gap",
    "session_total_sec", "session_cnt", 
    "step_sum", "cell_change_cnt", "wifi_change_cnt_est", # ✅ wifi_change_cnt_est로 업데이트
    "heartbeat_cnt", "retry_max", "queue_max",
]

for c in NUM_COLS:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

ZERO_FILL_COLS = ["daily_event_cnt", "unlock_cnt", "step_sum", "session_cnt", "heartbeat_cnt"]
for c in ZERO_FILL_COLS:
    if c in df.columns:
        df[c] = df[c].fillna(0)

# (활동이 0이면 gap_max는 이론상 24시간(86400초)이므로 최대값으로 채워주는 게 안전할 수 있음)
if "gap_max" in df.columns:
    df.loc[df["daily_event_cnt"] == 0, "gap_max"] = df["gap_max"].fillna(24.0)  # hours

# sanity quick check
print("daily_event_cnt range:", float(df["daily_event_cnt"].min()), float(df["daily_event_cnt"].max()))
print("has_activity nulls:", int(df["has_activity"].isna().sum()))
print("qc_core_low_cov mean:", float(df["qc_core_low_cov"].fillna(False).mean()))
print("travel_signal_raw range:", 
      float(df["travel_signal_raw"].min(skipna=True)), 
      float(df["travel_signal_raw"].max(skipna=True)))
if "gap_max" in df.columns:
    print("gap_max p99:", float(df["gap_max"].quantile(0.99)))

print("✅ type normalization done")

daily_event_cnt range: 0.0 2501.0
has_activity nulls: 0
qc_core_low_cov mean: 0.03919620183283648
travel_signal_raw range: 0.0 0.0
gap_max p99: 18.994685875000016
✅ type normalization done


# 2. 품질 게이트(Quality Gate) & 컨텍스트 라벨링

## 2.1 QC 플래그 표준화

In [5]:
QC_COLS = ["qc_core_low_cov", "qc_rhythm_low_cov", "qc_gap_low_cov"]

# 결측 감시 (정책: 결측이면 LOW_CONF로 내릴 것)
df["qc_missing_any"] = df[QC_COLS].isna().any(axis=1)

# QC 자체는 boolean + 결측은 False로 표준화 (연산 안정화)
for c in QC_COLS:
    df[c] = df[c].astype("boolean").fillna(False)

# meta qc (없으면 NA 유지해도 되지만, 운영상 boolean False로 두는 게 보통 더 편함)
META_QC_COLS = ["qc_meta_low_heartbeat", "qc_meta_retry_warn", "qc_meta_queue_warn"]
for c in META_QC_COLS:
    if c in df.columns:
        df[c] = df[c].astype("boolean")
    else:
        df[c] = pd.Series([pd.NA] * len(df), dtype="boolean")

df["qc_any_low_cov"] = df["qc_core_low_cov"] | df["qc_rhythm_low_cov"] | df["qc_gap_low_cov"]
print(f"✅ QC standardized. qc_missing_any: {df['qc_missing_any'].mean():.2%}")

✅ QC standardized. qc_missing_any: 0.00%


## 2.2 컨텍스트 시그널 → 플래그/레벨로 변환

In [6]:
df["partial_lv"] = pd.to_numeric(df["partial_signal_raw"], errors="coerce").fillna(0.0)
df["travel_lv"]  = pd.to_numeric(df["travel_signal_raw"],  errors="coerce").fillna(0.0)
df["tz_lv"]      = pd.to_numeric(df["tz_change_signal"],   errors="coerce").fillna(0.0)

PARTIAL_THR = 1.0
TRAVEL_THR  = 2.0
TZ_THR      = 1.0

df["partial_flag"] = (df["partial_lv"] >= PARTIAL_THR).astype("boolean")
df["tz_flag"]      = (df["tz_lv"]      >= TZ_THR).astype("boolean")

# travel: travel_lv OR tz_flag
df["travel_flag"]  = ((df["travel_lv"] >= TRAVEL_THR) | (df["tz_flag"])).astype("boolean")

## 2.3  quality_state 정의

In [7]:
qs = np.full(len(df), "GOOD", dtype=object)

# CRITICAL은 백엔드가 RDS에 직접 씀, ML은 판정 안 함
mask_lowcov = df["qc_any_low_cov"].to_numpy() | df["qc_missing_any"].to_numpy()
qs[mask_lowcov] = "LOW_CONF"
df["quality_state"] = qs

## 2.4 context_mode 정의

In [8]:
# 1. 개별 마스크 준비 (서로를 배제하지 않음)
m_partial = df["partial_flag"].fillna(False).to_numpy()
m_travel  = df["travel_flag"].fillna(False).to_numpy()

# 2. 분석/리포트용 통합 문자열 생성 (예: "TRAVEL|PARTIAL")
# 초기값은 NORMAL
cm = np.full(len(df), "NORMAL", dtype=object)

# 각 조건이 맞으면 문구를 추가 (우선순위 없이 모두 반영)
# 좀 더 깔끔한 문자열 관리를 위해 아래와 같이 처리할 수 있습니다.
temp_list = []
for i in range(len(df)):
    flags = []
    if m_partial[i]: flags.append("PARTIAL")
    if m_travel[i]:  flags.append("TRAVEL")
    
    if not flags:    temp_list.append("NORMAL")
    else:            temp_list.append("|".join(flags))

df["context_mode"] = temp_list

# 3. (옵션) 하나라도 특수 상황인지 여부 (나중에 로직 분기용)
df["context_any_warn"] = (m_partial | m_travel)

print("=== Context Mode Summary (v3) ===")
print(df["context_mode"].value_counts().head(10))

=== Context Mode Summary (v3) ===
context_mode
NORMAL     8979
PARTIAL      78
Name: count, dtype: int64


## 2.5 마스크 3종 정의

In [9]:
has_act = df["has_activity"].astype("boolean").fillna(False).to_numpy()

# 1) baseline_fit_mask: "순수 정상" 날만 학습 (수정 없음, 유지)
df["baseline_fit_mask"] = (
    has_act
    & (~df["qc_any_low_cov"].to_numpy())
    & (~df["qc_missing_any"].to_numpy())
    & (~df["partial_flag"].fillna(False).to_numpy())
    & (~df["travel_flag"].fillna(False).to_numpy())
)

# 2) z_score_eligible: 점수를 계산할 수 있는 날 (활동이 있고 치명적 오류가 없는 날)
# 이제 "여행 중"이라도 점수는 일단 계산합니다. 판단은 4번 섹션에서!
df["z_score_eligible"] = has_act

print("=== Gate Summary (v3 Independent) ===")
print(f"Baseline Fit Ratio: {df['baseline_fit_mask'].mean():.1%}")
print(f"Context Flags:")
print(f" - Partial: {df['partial_flag'].mean():.1%}")
print(f" - Travel:  {df['travel_flag'].mean():.1%}")
print(f" - Any Warn: {df['context_any_warn'].mean():.1%}")

=== Gate Summary (v3 Independent) ===
Baseline Fit Ratio: 96.1%
Context Flags:
 - Partial: 0.9%
 - Travel:  0.0%
 - Any Warn: 0.9%


# 3. Baseline 생성
## 3.1 Baseline 하이퍼파라미터

In [10]:
# windows
ST_WINDOW = 14
LT_WINDOW = 56

# min periods
ST_MINP_SPLIT  = 5     # split(평일/주말) 최소
ST_MINP_GLOBAL = 7     # split 부족할 때 global ST로
LT_MINP        = 14    # LT는 더 많이 필요

DEFAULT_FLOOR = 0.1   # ← 최소 0.05 이상

STD_FLOOR_MAP = {
    "daily_event_cnt": 10.0,
    "night_ratio": 0.05,
    "hour_entropy": 0.05,
    "Notif": 5.0,
    "UserAct": 5.0,
    "Screen": 3.0,
    "gap_max": 0.5,           
    "gap_cnt_2h": 1.0,
    "gap_long_ratio": 0.05,
    "overnight_gap": 0.5,     
    "gap_p95": 0.5,           
    "gap_cnt_6h": 0.5,        
    "session_total_sec": 60.0, 
    "session_cnt": 1.0,
    "step_sum": 50.0,         
}

# ✅ v3에서 Z에 넣을 베이스 피처 (원하면 여기서 조절)
BASE_COLS = [
    "Screen", "Notif", "UserAct",
    "daily_event_cnt", "night_ratio", "hour_entropy",
    "gap_max", "gap_cnt_2h", "gap_long_ratio", "overnight_gap",
    "gap_p95", "gap_cnt_6h", "session_total_sec", "session_cnt", "step_sum"
]
BASE_COLS = [c for c in BASE_COLS if c in df.columns]
if not BASE_COLS:
    raise ValueError("BASE_COLS is empty. Check df columns.")

# std floor map 채우기
for c in BASE_COLS:
    if c not in STD_FLOOR_MAP:
        STD_FLOOR_MAP[c] = DEFAULT_FLOOR

# 숫자형 강제
for c in BASE_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 정렬 보장
df = df.sort_values(["uuid", "date"]).reset_index(drop=True)

print("=== Baseline Config ===")
print("BASE_COLS:", BASE_COLS)
print("ST/LT:", (ST_WINDOW, LT_WINDOW), "MINP(split/global/LT):", (ST_MINP_SPLIT, ST_MINP_GLOBAL, LT_MINP))

=== Baseline Config ===
BASE_COLS: ['Screen', 'Notif', 'UserAct', 'daily_event_cnt', 'night_ratio', 'hour_entropy', 'gap_max', 'gap_cnt_2h', 'gap_long_ratio', 'overnight_gap', 'gap_p95', 'gap_cnt_6h']
ST/LT: (14, 56) MINP(split/global/LT): (5, 7, 14)


## 3.2 Baseline 학습 마스크

In [11]:
BASELINE_FIT_MASK = pd.Series(df["baseline_fit_mask"]).astype("boolean").fillna(False)

print("baseline_fit_mask ratio:", round(BASELINE_FIT_MASK.mean(), 3))

# (옵션) 유저별 fit 일수 요약
tmp = df.loc[BASELINE_FIT_MASK].groupby("uuid")["date"].nunique()

if tmp.empty:
    print("⚠️ baseline_fit_mask True rows = 0 (no days to fit baseline).")
else:
    print("baseline fit days per user (min/median/max):",
          int(tmp.min()), int(tmp.median()), int(tmp.max()))


baseline_fit_mask ratio: 0.961
baseline fit days per user (min/median/max): 6 26 77


## 3.3 LT(Global) baseline

In [12]:
# [v3] Winsorize helper: baseline 학습 전 극단값 클리핑
def winsorize_series(s: pd.Series, lower_q=0.005, upper_q=0.995) -> pd.Series:
    """유효한 값 기준으로 quantile clip (NaN 보존)"""
    valid = s.dropna()
    if len(valid) < 10:
        return s
    lo = float(valid.quantile(lower_q))
    hi = float(valid.quantile(upper_q))
    return s.clip(lower=lo, upper=hi)

for c in BASE_COLS:
    x = pd.to_numeric(df[c], errors="coerce")

    # fit mask 밖은 NaN 처리 후, "오늘"은 쓰면 안 되니 shift(1)
    x_fit = x.where(BASELINE_FIT_MASK, np.nan)
    # [v3] Winsorize: 극단값이 rolling mean/std를 오염하지 않도록
    x_fit = winsorize_series(x_fit)
    x_shift = x_fit.groupby(df["uuid"], sort=False).shift(1)

    roll = x_shift.groupby(df["uuid"], sort=False).rolling(LT_WINDOW, min_periods=LT_MINP)

    mean_lt = roll.mean().reset_index(level=0, drop=True).sort_index()
    std_lt  = roll.std(ddof=0).reset_index(level=0, drop=True).sort_index()

    floor = float(STD_FLOOR_MAP.get(c, DEFAULT_FLOOR))
    std_lt = std_lt.replace(0, np.nan).fillna(floor).clip(lower=floor)

    df[f"{c}_mean_lt_g"] = mean_lt
    df[f"{c}_std_lt_g"]  = std_lt

print("✅ LT baseline added:", [f"{c}_mean_lt_g" for c in BASE_COLS][:5], "...")

✅ LT baseline added: ['Screen_mean_lt_g', 'Notif_mean_lt_g', 'UserAct_mean_lt_g', 'daily_event_cnt_mean_lt_g', 'night_ratio_mean_lt_g'] ...


## 3.4 ST baseline (오늘 판단 기준선)

In [13]:
# 3.4.1 ST Global 먼저 만들기 (split이 부족할 때 fallback용)
for c in BASE_COLS:
    x = pd.to_numeric(df[c], errors="coerce")
    x_fit = x.where(BASELINE_FIT_MASK, np.nan)
    x_fit = winsorize_series(x_fit)  # [v3]
    x_shift = x_fit.groupby(df["uuid"], sort=False).shift(1)

    roll = x_shift.groupby(df["uuid"], sort=False).rolling(ST_WINDOW, min_periods=ST_MINP_GLOBAL)
    mean_st_g = roll.mean().reset_index(level=0, drop=True).sort_index()
    std_st_g  = roll.std(ddof=0).reset_index(level=0, drop=True).sort_index()

    floor = float(STD_FLOOR_MAP.get(c, DEFAULT_FLOOR))
    std_st_g = std_st_g.replace(0, np.nan).fillna(floor).clip(lower=floor)

    df[f"{c}_mean_st_g"] = mean_st_g
    df[f"{c}_std_st_g"]  = std_st_g

# 3.4.2 ST Split 생성 + fallback 체인
fallback_flags = []

for c in BASE_COLS:
    x = pd.to_numeric(df[c], errors="coerce")
    x_fit = x.where(BASELINE_FIT_MASK, np.nan)
    x_fit = winsorize_series(x_fit)  # [v3]

    grp = [df["uuid"], df["is_weekend"]]
    x_shifted = x_fit.groupby(grp, sort=False).shift(1)

    roll = x_shifted.groupby(grp, sort=False).rolling(ST_WINDOW, min_periods=ST_MINP_SPLIT)

    mean_st_s_raw = roll.mean().reset_index(level=[0,1], drop=True).sort_index()
    std_st_s_raw  = roll.std(ddof=0).reset_index(level=[0,1], drop=True).sort_index()

    floor = float(STD_FLOOR_MAP.get(c, DEFAULT_FLOOR))
    std_st_s_raw = std_st_s_raw.replace(0, np.nan).fillna(floor).clip(lower=floor)

    # fallback chain
    mean_1 = mean_st_s_raw.fillna(df[f"{c}_mean_st_g"])
    std_1  = std_st_s_raw.fillna(df[f"{c}_std_st_g"])

    mean_final = mean_1.fillna(df[f"{c}_mean_lt_g"])
    std_final  = std_1.fillna(df[f"{c}_std_lt_g"])

    df[f"{c}_mean_st_s"] = mean_final
    df[f"{c}_std_st_s"]  = std_final

    flag_col = f"{c}_fallback_stsplit"
    df[flag_col] = mean_st_s_raw.isna()
    fallback_flags.append(flag_col)

df["fallback_any_stsplit"] = df[fallback_flags].any(axis=1)
print("✅ ST split baseline + fallback done.")
print("fallback_any_stsplit ratio:", round(float(df["fallback_any_stsplit"].mean()), 3))


✅ ST split baseline + fallback done.
fallback_any_stsplit ratio: 0.425


## 3.5 baseline_ready 정의

In [14]:
mean_cols_st = [f"{c}_mean_st_s" for c in BASE_COLS]
std_cols_st  = [f"{c}_std_st_s"  for c in BASE_COLS]

# 모든 필수 피처의 평균/표준편차가 존재해야 정식 베이스라인 가동
# .all(axis=1)을 통해 하나라도 결측이면 False 처리 (엄격한 기준)
df["baseline_ready"] = df[mean_cols_st + std_cols_st].notna().all(axis=1)

print("=== Baseline Ready Summary (v3) ===")
print(f"✅ baseline_ready ratio: {df['baseline_ready'].mean():.1%}")

# 유저별 정식 베이스라인 진입 시점 분석
first_ready = df.loc[df["baseline_ready"]].groupby("uuid")["date"].min()

if not first_ready.empty:
    # 날짜 출력 시 .date()를 붙여 시분초를 생략하면 더 깔끔합니다.
    print(f"📅 first baseline_ready day:\n - Min: {first_ready.min().date()}\n - Med: {first_ready.median()}\n - Max: {first_ready.max().date()}")
else:
    missing_info = df[mean_cols_st + std_cols_st].isna().sum()
    print("⚠️ No baseline_ready days found. Check missing counts:")
    print(missing_info[missing_info > 0].head(5))

# 메모리 조각화 방지
df = df.copy()

=== Baseline Ready Summary (v3) ===
✅ baseline_ready ratio: 68.6%
📅 first baseline_ready day:
 - Min: 2016-04-22
 - Med: 2016-06-20 00:00:00
 - Max: 2016-07-09


## 3.6 콜드스타트용 EARLY 베이스라인 만들기 (minp=3)

In [15]:
EARLY_WINDOW = 7
EARLY_MINP_GLOBAL = 3  # <- 너 목표(3일 지나면) 반영
EARLY_MINP_SPLIT  = 3

# baseline 재료는 기존 BASELINE_FIT_MASK 그대로 사용(깨끗한 날)
fit_mask = BASELINE_FIT_MASK

for c in BASE_COLS:
    x = df[c].where(fit_mask, np.nan)
    x = winsorize_series(x)  # [v3]
    x_shifted = x.groupby(df["uuid"], sort=False).shift(1)

    # Global Early (sort_index 추가)
    roll = x_shifted.groupby(df["uuid"], sort=False).rolling(EARLY_WINDOW, min_periods=EARLY_MINP_GLOBAL)
    mu_g = roll.mean().reset_index(level=0, drop=True).sort_index()
    sd_g = roll.std(ddof=0).reset_index(level=0, drop=True).sort_index()

    floor = STD_FLOOR_MAP.get(c, DEFAULT_FLOOR)
    sd_g = sd_g.replace(0, np.nan).fillna(floor).clip(lower=floor)

    df[f"{c}_mean_early_g"] = mu_g
    df[f"{c}_std_early_g"]  = sd_g

    # split early (weekend/weekday) + fallback to early global
    grp = [df["uuid"], df["is_weekend"]]
    x_shifted_2 = x.groupby(grp, sort=False).shift(1)
    
    roll2 = x_shifted_2.groupby(grp, sort=False).rolling(EARLY_WINDOW, min_periods=EARLY_MINP_SPLIT)
    mu_s = roll2.mean().reset_index(level=[0,1], drop=True).sort_index()
    sd_s = roll2.std(ddof=0).reset_index(level=[0,1], drop=True).sort_index()

    sd_s = sd_s.replace(0, np.nan).fillna(floor).clip(lower=floor)

    df[f"{c}_mean_early_s"] = mu_s.fillna(df[f"{c}_mean_early_g"])
    df[f"{c}_std_early_s"]  = sd_s.fillna(df[f"{c}_std_early_g"])

# early_ready: early baseline이 준비된 날
early_mean_cols = [f"{c}_mean_early_s" for c in BASE_COLS]
early_std_cols  = [f"{c}_std_early_s"  for c in BASE_COLS]
df["early_ready"] = df[early_mean_cols + early_std_cols].notna().all(axis=1)

print("early_ready ratio:", round(float(df["early_ready"].mean()), 3))
df = df.copy()

early_ready ratio: 0.834


## 3.7 최종 베이스라인 통합

In [16]:
# 정식(ST)이 있으면 쓰고, 없으면 조기(Early) 버전 사용
for c in BASE_COLS:
    df[f"{c}_mean_final"] = df[f"{c}_mean_st_s"].fillna(df[f"{c}_mean_early_s"])
    df[f"{c}_std_final"]  = df[f"{c}_std_st_s"].fillna(df[f"{c}_std_early_s"])

mean_final_cols = [f"{c}_mean_final" for c in BASE_COLS]
std_final_cols  = [f"{c}_std_final"  for c in BASE_COLS]

df["analysis_ready"] = df[mean_final_cols + std_final_cols].notna().all(axis=1)

print(f"✅ Final analysis_ready ratio: {df['analysis_ready'].mean():.1%}")

✅ Final analysis_ready ratio: 83.8%


# 4. Z-Score 생성
## 4.0 설정

In [17]:
Z_CLIP = 4.0  # v1과 동일 (안정성 우선)

# v3에서 Z 만들 대상 피처(있는 것만 자동 선택)
Z_BASE_CANDIDATES = [
    "daily_event_cnt",
    # core breakdown (FEv3)
    "Screen", "Notif", "UserAct",
    # rhythm
    "night_ratio", "hour_entropy",
    # gap/inactivity
    "gap_max", "gap_p95", "gap_cnt_2h", "gap_cnt_6h", "gap_long_ratio", "overnight_gap",
    # session
    "session_total_sec", "session_cnt",
    # mobility
    "step_sum",
]

## 4.1 Baseline suffix 결정 (ST 기준)

In [18]:
MEAN_SUFFIX_ST = "_mean_st_s"
STD_SUFFIX_ST  = "_std_st_s"

## 4.2 Z-score 생성 마스크 정의

In [19]:
if "baseline_ready" not in df.columns:
    raise ValueError("[Z] baseline_ready missing. Run section 3 (Baseline) first.")

if "quality_state" not in df.columns:
    raise ValueError("[Z] quality_state missing. Run section 2 (Gate) first.")

# (quality_state가 PD 때문에 LOW_CONF로 내려가지 않으므로)
# NORMAL / PARTIAL / TRAVEL 모두 z_mask 대상이 될 수 있음 (baseline_ready일 때)
z_mask = (df["baseline_ready"].fillna(False)) & (df["quality_state"] == "GOOD")
print("z_mask ratio:", float(z_mask.mean()))


z_mask ratio: 0.665120900960583


## 4.3 사용할 피처/컬럼 자동 필터링

In [20]:
Z_BASE_COLS = [c for c in Z_BASE_CANDIDATES if c in df.columns]

if len(Z_BASE_COLS) == 0:
    raise ValueError("[Z] No Z_BASE_COLS found in df. Check v3 FE output columns.")

# baseline 컬럼 존재 체크 (없으면 그 피처는 Z에서 제외)
usable = []
missing_baseline = []

for c in Z_BASE_COLS:
    mean_col = f"{c}{MEAN_SUFFIX_ST}"
    std_col  = f"{c}{STD_SUFFIX_ST}"
    if (mean_col in df.columns) and (std_col in df.columns):
        usable.append(c)
    else:
        missing_baseline.append((c, mean_col, std_col))

if len(usable) == 0:
    raise ValueError(
        "[Z] None of candidate features have ST baseline columns.\n"
        f"Example missing: {missing_baseline[:5]}"
    )

if missing_baseline:
    print("⚠️ [Z] Some features skipped (missing baseline cols). Examples:")
    for row in missing_baseline[:8]:
        print("   -", row)

Z_BASE_COLS = usable
print("✅ Z will be computed for:", Z_BASE_COLS)

✅ Z will be computed for: ['daily_event_cnt', 'Screen', 'Notif', 'UserAct', 'night_ratio', 'hour_entropy', 'gap_max', 'gap_p95', 'gap_cnt_2h', 'gap_cnt_6h', 'gap_long_ratio', 'overnight_gap']


## 4.4 Z-score 계산

In [21]:
Z_COLS = []

for c in Z_BASE_COLS:
    mean_col = f"{c}{MEAN_SUFFIX_ST}"
    std_col  = f"{c}{STD_SUFFIX_ST}"
    z_col    = f"{c}_z"

    # 안전 변환
    x = pd.to_numeric(df[c], errors="coerce")
    mu = pd.to_numeric(df[mean_col], errors="coerce")
    sd = pd.to_numeric(df[std_col], errors="coerce")

    z = (x - mu) / sd
    z = z.replace([np.inf, -np.inf], np.nan)

    # soft clip (tanh)
    z = Z_CLIP * np.tanh(z / Z_CLIP)

    # 핵심: 이제 이 z값은 TRAVEL/PARTIAL 상황의 수치도 담고 있습니다.
    df[z_col] = z.where(z_mask, np.nan)

    Z_COLS.append(z_col)

print("✅ Z-score columns added (Context-inclusive):", Z_COLS)
df = df.copy()

✅ Z-score columns added (Context-inclusive): ['daily_event_cnt_z', 'Screen_z', 'Notif_z', 'UserAct_z', 'night_ratio_z', 'hour_entropy_z', 'gap_max_z', 'gap_p95_z', 'gap_cnt_2h_z', 'gap_cnt_6h_z', 'gap_long_ratio_z', 'overnight_gap_z']


# 5. IsolationForest MODEL

## 5.1 입력 피처 결정

In [22]:
from sklearn.ensemble import IsolationForest

assert "Z_COLS" in globals() or "Z_COLS" in locals(), "Z_COLS not found. Run section 4 first."
assert "z_mask" in globals() or "z_mask" in locals(), "z_mask not found. Run section 4 first."

# Z 컬럼 중 결측 너무 많은 건 제외(옵션)
Z_NA_THR = 0.60  # 60% 이상 결측이면 드랍 (너 상황에선 대부분 동일 NA라 큰 의미는 없지만 안전장치)
z_na_ratio = df[Z_COLS].isna().mean()
Z_USE = [c for c in Z_COLS if z_na_ratio[c] < Z_NA_THR]

if len(Z_USE) < 3:
    # 너무 적으면 그냥 원래대로
    Z_USE = Z_COLS.copy()

print("=== Anomaly Config ===")
print("n(Z_COLS):", len(Z_COLS), " -> n(Z_USE):", len(Z_USE))
print("Top NA Z (desc):")
print(z_na_ratio.sort_values(ascending=False).head(8).round(3))

=== Anomaly Config ===
n(Z_COLS): 12  -> n(Z_USE): 12
Top NA Z (desc):
overnight_gap_z      0.339
daily_event_cnt_z    0.335
Notif_z              0.335
Screen_z             0.335
UserAct_z            0.335
night_ratio_z        0.335
gap_max_z            0.335
hour_entropy_z       0.335
dtype: float64


## 5.2 학습/스코어 마스크 정의

In [23]:
# (A) train_mask: IForest는 "정상 분포"를 학습해야 해서 최대한 깨끗한 날로만
# - baseline_fit_mask: travel/partial/lowcov 제거된 날
# - baseline_ready: ST split baseline이 준비된 날
# - quality_state == GOOD: 최종적으로 신뢰 가능한 날
#
# 참고: baseline_fit_mask는 오늘 값 포함 가능한데, IForest 학습은 "행동 자체"를 학습하므로
# leakage 개념이 덜 민감함. 그래도 안전하게 "오늘 판단 가능한 날"로만 학습하자.
train_mask = (
    df["baseline_fit_mask"].fillna(False)
    & df["baseline_ready"].fillna(False)
    & (df["quality_state"] == "GOOD")
)

score_mask = z_mask.copy()

print("\n=== Mask Summary ===")
print("train_mask ratio:", round(float(train_mask.mean()), 3))
print("score_mask ratio:", round(float(score_mask.mean()), 3))


=== Mask Summary ===
train_mask ratio: 0.665
score_mask ratio: 0.665


## 5.3 학습 데이터 구성

In [24]:
# IForest는 NaN 못 받음 -> 결측 처리 필요
# 옵션 1) 0 대체: Z-score의 0은 '평소와 동일' 의미라 가장 직관적/안전
# 옵션 2) median 대체: 피처별 중앙값
#
# 여기선 옵션 1(0 대체) 추천. 왜? 우리 Z는 baseline 기반이라 0이 의미 있음.
X_train = df.loc[train_mask, Z_USE].copy()
X_score = df.loc[score_mask, Z_USE].copy()

# 결측 -> 0
X_train = X_train.fillna(0.0)
X_score = X_score.fillna(0.0)

print("\nX_train shape:", X_train.shape, "X_score shape:", X_score.shape)


X_train shape: (6024, 12) X_score shape: (6024, 12)


## 5.4 IsolationForest 학습

In [25]:
IF_CONTAM = 0.05
IF_ESTIMATORS = 300
IF_MAX_SAMPLES = "auto"
IF_SEED = 42

if len(X_train) < 200:
    print("⚠️ X_train too small. Consider relaxing train_mask or lowering MINP/windows.")
    
iso = IsolationForest(
    n_estimators=IF_ESTIMATORS,
    contamination=IF_CONTAM,
    max_samples=IF_MAX_SAMPLES,
    random_state=IF_SEED,
    n_jobs=-1,
)

iso.fit(X_train)

,n_estimators,300
,max_samples,'auto'
,contamination,0.05
,max_features,1.0
,bootstrap,False
,n_jobs,-1
,random_state,42
,verbose,0
,warm_start,False


## 5.5 anomaly score 생성

In [26]:
# sklearn convention:
# - decision_function: 높을수록 정상(positive)
# - score_samples: 높을수록 정상(less abnormal)
# 우리는 "높을수록 이상"으로 바꿔서 쓰자.
#
# raw_abnormal = -decision_function
raw = -iso.decision_function(X_score)  # higher = more anomalous

# df에 저장 (score_mask 아닌 날은 NaN)
df["anomaly_score_raw"] = np.nan
df.loc[score_mask, "anomaly_score_raw"] = raw

## 5.6 스케일링(0~1) + 안정화

In [27]:
# 5.6 스케일링(0~1) - PATCH v3: per-user quantile scaling (tie에 강함)

Q_LOW  = 0.80   # 80% 이하는 0 근처
Q_HIGH = 0.99   # 99%는 1 근처
FLAG_Q = 0.99   # flag 기준

df["anomaly_score"] = np.nan
df["anomaly_flag"]  = pd.Series([pd.NA] * len(df), dtype="boolean")

tmp = df.loc[score_mask, ["uuid", "anomaly_score_raw"]].copy()

# 유저별 q80, q99 계산
q = tmp.groupby("uuid")["anomaly_score_raw"].quantile([Q_LOW, Q_HIGH]).unstack()
q.columns = ["q_low", "q_high"]
tmp = tmp.join(q, on="uuid")

den = (tmp["q_high"] - tmp["q_low"]).replace(0, np.nan)

# 0~1로 선형 매핑 + clip
tmp["anomaly_score"] = ((tmp["anomaly_score_raw"] - tmp["q_low"]) / den).clip(0, 1).fillna(0.0)

# flag는 q_high 이상
tmp["anomaly_flag"] = (tmp["anomaly_score_raw"] >= tmp["q_high"]).astype("boolean")

df.loc[score_mask, "anomaly_score"] = tmp["anomaly_score"].values
df.loc[score_mask, "anomaly_flag"]  = tmp["anomaly_flag"].values

print("\n=== Anomaly Summary (PATCH v3: per-user quantile scaling) ===")
print(df.loc[score_mask, "anomaly_score"].describe().round(4))
print("anomaly_flag rate:",
      round(float(df.loc[score_mask, "anomaly_flag"].astype(float).mean()), 3))


=== Anomaly Summary (PATCH v3: per-user quantile scaling) ===
count    6024.0000
mean        0.1273
std         0.2963
min         0.0000
25%         0.0000
50%         0.0000
75%         0.0000
max         1.0000
Name: anomaly_score, dtype: float64
anomaly_flag rate: 0.056


# 6. Risk Score

## 6.1 입력 컬럼/리스트 검증

In [28]:
REQ_COLS = ["uuid", "date", "quality_state", "context_mode", "baseline_ready", "anomaly_score"]
missing = [c for c in REQ_COLS if c not in df.columns]
if missing:
    raise ValueError(f"[Risk] Missing columns: {missing}")

# Z 컬럼 리스트
if "Z_COLS" not in globals():
    Z_COLS = [c for c in df.columns if c.endswith("_z") and (not c.endswith("_z_early"))]
if len(Z_COLS) == 0:
    raise ValueError("[Risk] Z_COLS empty. Run step 4 first.")

print("=== Risk Inputs ===")
print("n(Z):", len(Z_COLS))

=== Risk Inputs ===
n(Z): 12


## 6.2 리스크 산출 마스크

In [29]:
# 필수 컬럼 체크
need = ["quality_state", "baseline_ready", "qc_any_low_cov"]
missing = [c for c in need if c not in df.columns]
if missing:
    raise ValueError(f"[6.2] Missing columns: {missing}")

# GOOD + baseline_ready + (추가) low_cov 제외
df["risk_mask"] = (
    (df["quality_state"] == "GOOD")
    & (df["baseline_ready"].fillna(False))
    & (~df["qc_any_low_cov"].fillna(False))
    & (~df["qc_missing_any"].fillna(False))   # ✅ QC 결측 보수적 제외(추천)
)
risk_mask = df["risk_mask"].astype(bool)
print("[6.2] risk_mask ratio:", round(float(risk_mask.mean()), 3))

[6.2] risk_mask ratio: 0.665


## 6.3 컴포넌트

In [30]:
Z_ABS = df[Z_COLS].abs()

def _trim_mean(row, keep_ratio=0.8):
    s = row.dropna().sort_values()
    if len(s) == 0:
        return np.nan
    k = max(1, int(len(s) * keep_ratio))
    return float(s.iloc[:k].mean())

Z_ABS_N = Z_ABS

# ✅ (B) Z 그룹 정의
Z_GROUPS = {
    "core":   [c for c in ["Screen_z", "Notif_z", "UserAct_z", "daily_event_cnt_z"] if c in Z_COLS],
    "rhythm": [c for c in ["night_ratio_z", "hour_entropy_z"] if c in Z_COLS],
    "gap":    [c for c in ["gap_max_z", "gap_p95_z", "gap_cnt_2h_z", "gap_cnt_6h_z", "gap_long_ratio_z", "overnight_gap_z"] if c in Z_COLS],
    "session":[c for c in ["session_total_sec_z", "session_cnt_z"] if c in Z_COLS],
    "mob":    [c for c in ["step_sum_z"] if c in Z_COLS],
}

# 빈 그룹 제거
Z_GROUPS = {k:v for k,v in Z_GROUPS.items() if len(v) > 0}
if len(Z_GROUPS) == 0:
    raise ValueError("[6.3] Z_GROUPS became empty. Check Z_COLS naming.")

# ✅ (C) 그룹 대표 스코어 계산
def _row_quantile(df_part: pd.DataFrame, q=0.9):
    # row-wise quantile
    return df_part.quantile(q=q, axis=1, interpolation="linear")

CORE_Q = 0.85
OTHER_Q = 0.90

for gname, cols in Z_GROUPS.items():
    q = CORE_Q if gname == "core" else OTHER_Q
    df[f"z_{gname}_rep"] = Z_ABS_N[cols].quantile(q=q, axis=1, interpolation="linear")

# ✅ (D) 최종 z_agg: 그룹 대표값들의 trim-mean or mean
group_rep_cols = [f"z_{g}_rep" for g in Z_GROUPS.keys()]
df["z_group_trim_mean"] = df[group_rep_cols].apply(_trim_mean, axis=1)
df["z_trim_mean_abs"] = df["z_group_trim_mean"]

# 진단용
df["z_max_abs"] = Z_ABS_N.max(axis=1)  # 전체 피처 중 최대(기존 유지)
df["z_trim_mean_abs_raw"] = Z_ABS_N.apply(_trim_mean, axis=1)  # (참고) 기존 방식 값도 남겨보고 싶으면

# ✅ (E) Soft squash (simple) - 과한 CAP/GAMMA 제거 (FIXED)
ref = df.loc[risk_mask, "z_group_trim_mean"].dropna()

TAU_Q = 0.90
TAU = float(ref.quantile(TAU_Q)) if len(ref) else 1.0
TAU = max(TAU, 1e-6)

# exp squash 1방: 0~1
df["z_score"] = (1.0 - np.exp(-df["z_group_trim_mean"] / (TAU + 1e-6))).clip(0, 0.999)

# -------------------------
# A part: anomaly_score -> a_score (기존 그대로)
# -------------------------
if "anomaly_score" not in df.columns:
    raise ValueError("[6.3][A] anomaly_score missing. Run section 5 first.")

df["anomaly_score"] = pd.to_numeric(df["anomaly_score"], errors="coerce").clip(0, 1)

# 6.3 A part - PATCH: a_score smoother
df["a_score"] = df["anomaly_score"].clip(0, 0.999)



## 6.4 결합 리스크 (risk_pre)

In [31]:
W_Z = 0.85
W_A = 0.15

df["risk_pre"] = (W_Z * df["z_score"] + W_A * df["a_score"]).clip(0, 1)

## 6.5 상황 보정 (risk_adj)

In [32]:
DISCOUNT = {
    "NORMAL": 1.00,
    "PARTIAL": 0.60,
    "TRAVEL": 0.70,
}

# [v3] context_mode가 "PARTIAL|TRAVEL" 같은 복합 문자열일 수 있으므로
# 포함된 모든 컨텍스트 중 가장 낮은(보수적인) discount를 적용
def _resolve_discount(cm: str) -> float:
    if pd.isna(cm) or cm == "NORMAL":
        return 1.0
    parts = str(cm).split("|")
    vals = [DISCOUNT.get(p.strip(), 1.0) for p in parts]
    return min(vals)  # 가장 보수적인(낮은) discount 적용

df["risk_adj"] = (df["risk_pre"] * df["context_mode"].apply(_resolve_discount)).astype(float)

# risk_mask 밖 제외
df.loc[~risk_mask, ["risk_pre", "risk_adj"]] = np.nan

## 6.6 EMA Smoothing

In [33]:
EMA_ALPHA = 0.25  # 0.20~0.30 추천
df["risk_score"] = (
    df.groupby("uuid", sort=False)["risk_adj"]
      .transform(lambda s: s.ewm(alpha=EMA_ALPHA, adjust=False, ignore_na=True).mean())
)

df.loc[~risk_mask, "risk_score"] = np.nan
df = df.copy()

## 6.7 Recovery Gate

In [34]:
# [v3] recovery_ok/recovery_block 제거
# stable_hold_flag가 같은 역할을 더 정확하게 수행하므로 중복 코드 정리
# (stable_hold_flag는 셀 72에서 bad_history_flag 기반으로 생성)

df = df.sort_values(["uuid", "date"]).reset_index(drop=True)
print("✅ Recovery Gate: stable_hold_flag로 통합 (recovery_ok/block 제거)")


✅ Recovery Gate: stable_hold_flag로 통합 (recovery_ok/block 제거)


## 6.8 early_risk + final_risk

In [35]:
early_mask = (
    (df["quality_state"] == "GOOD")
    & (df["cold_stage"].isin(["WARMUP", "SEMI_READY"]))
    & (df["early_ready"].fillna(False))
    & (~df["qc_any_low_cov"].fillna(False))
    & (~df["qc_missing_any"].fillna(False))
)

Z_CLIP_EARLY = 4.0

early_z_cols = []
for c in BASE_COLS:
    mu = df[f"{c}_mean_early_s"]
    sd = df[f"{c}_std_early_s"]
    z = (df[c] - mu) / sd
    z = z.replace([np.inf, -np.inf], np.nan)
    z = Z_CLIP_EARLY * np.tanh(z / Z_CLIP_EARLY)
    col = f"{c}_z_early"
    df[col] = z.where(early_mask, np.nan)
    early_z_cols.append(col)

def _trim_mean(row, keep_ratio=0.8):
    s = row.dropna().sort_values()
    if len(s) == 0:
        return np.nan
    k = max(1, int(len(s) * keep_ratio))
    return float(s.iloc[:k].mean())

early_ref_med = df.loc[early_mask, early_z_cols].abs().median()
early_ref_med = early_ref_med.replace(0, np.nan).fillna(1.0)

abs_early_norm = df[early_z_cols].abs().div(early_ref_med, axis=1)
df["z_trim_mean_abs_early"] = abs_early_norm.apply(_trim_mean, axis=1)

ref_e = df.loc[early_mask, "z_trim_mean_abs_early"].dropna()
TAU_E = float(ref_e.quantile(0.90)) if len(ref_e) else 1.0
TAU_E = max(TAU_E, 1e-6)

raw_e = 1.0 - np.exp(-df["z_trim_mean_abs_early"] / (TAU_E + 1e-6))
raw_e_max = 1.0 - np.exp(-(Z_CLIP_EARLY) / (TAU_E + 1e-6))

df["early_risk"] = np.nan
df.loc[early_mask, "early_risk"] = (raw_e / (raw_e_max + 1e-6)).clip(0, 1)

print("early_risk valid ratio:", round(float(df["early_risk"].notna().mean()), 3))

# final_risk = full risk 우선, 없으면 early_risk
df["final_risk"] = df["risk_score"]
df.loc[df["final_risk"].isna(), "final_risk"] = df["early_risk"]

# -------------------------
# Recovery policy -> HOLD flag only (risk 값은 왜곡하지 않음)
# -------------------------
HOLD_WINDOW = 14        # 최근 2주 히스토리
HOLD_MINP   = 7
BAD_THR     = 0.70      # "최근에 꽤 안좋았다" 기준
STABLE_THR  = 0.45      # 안정으로 내려갈 수 있는 위험수준

df = df.sort_values(["uuid", "date"]).reset_index(drop=True)

# 최근 HOLD_WINDOW에서 BAD_THR 이상이 있었는지 (히스토리)
df["bad_history_flag"] = (
    df.groupby("uuid", sort=False)["final_risk"]
      .transform(lambda s: s.shift(1).rolling(HOLD_WINDOW, min_periods=HOLD_MINP).max() >= BAD_THR)
).astype("boolean")

# 지금 risk가 STABLE_THR 아래로 내려가도, bad_history가 있으면 STABLE로 내리지 말자
df["stable_hold_flag"] = (
    df["bad_history_flag"].fillna(False)
    & df["final_risk"].notna()
    & (df["final_risk"] < STABLE_THR)
).astype("boolean")

# -------------------------
# [v3] Drift Risk: baseline이 서서히 나빠지는 것 감지
# -------------------------
# 원리: 초기 안정기(3~4주차) baseline을 앵커로 저장하고,
# 현재 baseline이 앵커 대비 크게 벗어나면 drift_flag = True
#
# 핵심 피처만 감시 (활동량, 수면패턴, 무활동)
DRIFT_WATCH_COLS = ["daily_event_cnt", "night_ratio", "gap_max"]
DRIFT_WATCH_COLS = [c for c in DRIFT_WATCH_COLS if c in BASE_COLS]

# 앵커 구간: day_idx 16~30 (READY 초기, baseline이 안정된 구간)
ANCHOR_START = 16
ANCHOR_END = 30
DRIFT_Z_THR = 2.0  # 앵커 대비 Z 2.0 이상 벗어나면 drift

anchor_mask = df["day_idx"].between(ANCHOR_START, ANCHOR_END) & df["baseline_fit_mask"]

for c in DRIFT_WATCH_COLS:
    # 앵커 구간의 유저별 mean/std 계산
    anchor_stats = (
        df.loc[anchor_mask]
        .groupby("uuid")[c]
        .agg(["mean", "std"])
        .rename(columns={"mean": f"{c}_anchor_mean", "std": f"{c}_anchor_std"})
    )
    # std floor 적용
    floor = STD_FLOOR_MAP.get(c, DEFAULT_FLOOR)
    anchor_stats[f"{c}_anchor_std"] = anchor_stats[f"{c}_anchor_std"].clip(lower=floor)
    
    df = df.join(anchor_stats, on="uuid")
    
    # 현재 baseline(ST) vs 앵커 비교
    mean_col = f"{c}_mean_final"
    if mean_col in df.columns:
        drift_z = (df[mean_col] - df[f"{c}_anchor_mean"]) / df[f"{c}_anchor_std"]
        df[f"{c}_drift_z"] = drift_z
    else:
        df[f"{c}_drift_z"] = np.nan

# drift_flag: 감시 피처 중 하나라도 drift 감지
drift_z_cols = [f"{c}_drift_z" for c in DRIFT_WATCH_COLS]
drift_z_abs = df[drift_z_cols].abs()

# 앵커가 준비된 날(day_idx > ANCHOR_END)에서만 판정
df["drift_flag"] = (
    (df["day_idx"] > ANCHOR_END)
    & (drift_z_abs.max(axis=1) >= DRIFT_Z_THR)
).astype("boolean").fillna(False)

# drift 중 어떤 피처가 문제인지 기록
df["drift_top_feature"] = ""
drift_valid = df["drift_flag"].fillna(False)
if drift_valid.any():
    df.loc[drift_valid, "drift_top_feature"] = drift_z_abs.loc[drift_valid].idxmax(axis=1).str.replace("_drift_z", "")

print(f"\n✅ Drift Risk 추가")
print(f"drift_flag rate (day>{ANCHOR_END}): {df.loc[df['day_idx']>ANCHOR_END, 'drift_flag'].mean():.1%}")
print(f"drift_top_feature dist:")
print(df.loc[drift_valid, "drift_top_feature"].value_counts().head())


early_risk valid ratio: 0.385

✅ Drift Risk 추가
drift_flag rate (day>30): 15.7%
drift_top_feature dist:
drift_top_feature
daily_event_cnt    90
Name: count, dtype: int64


# 7. 파일 저장

In [36]:
from pathlib import Path

# --- (추가) Z 컬럼 export에 포함 ---
Z_EXPORT = [c for c in df.columns if c.endswith("_z") and (not c.endswith("_z_early"))]
Z_EARLY_EXPORT = [c for c in df.columns if c.endswith("_z_early")]

print("Z_EXPORT:", len(Z_EXPORT), "e.g.", Z_EXPORT[:10])
print("Z_EARLY_EXPORT:", len(Z_EARLY_EXPORT), "e.g.", Z_EARLY_EXPORT[:10])

# 저장 경로
OUT_DIR = Path("../artifacts/model")
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_PATH = OUT_DIR / "model_out_v3.csv"

# Decoder / Backend에서 쓸 최소 컬럼만 추리기
OUT_COLS = [
    "uuid","date","day_idx","cold_stage",
    "early_ready","early_risk","final_risk",
    "quality_state","context_mode","baseline_ready",
    "risk_score","risk_pre","risk_adj",
    "z_score","a_score",
    "z_trim_mean_abs","z_max_abs",
    "anomaly_flag","anomaly_score",
    "partial_flag","travel_flag","tz_flag",
    "bad_history_flag","stable_hold_flag","drift_flag","drift_top_feature",
]

# ✅ 여기! z 컬럼을 export 리스트에 추가
OUT_COLS += Z_EXPORT  # Top-Z 디코더용 (필수)

# [v3] z_group_rep 컬럼도 export (Decoder에서 그룹별 판정에 사용)
Z_GROUP_REP_COLS = [c for c in df.columns if c.startswith("z_") and c.endswith("_rep")]
OUT_COLS += Z_GROUP_REP_COLS
# (선택) early z도 같이 내보내면 디코더에서 early reason까지 만들기 쉬움
# OUT_COLS += Z_EARLY_EXPORT

# 실제 존재하는 컬럼만 + 중복 제거
OUT_COLS = [c for c in OUT_COLS if c in df.columns]
OUT_COLS = list(dict.fromkeys(OUT_COLS))

print("Export columns:", len(OUT_COLS))
print("First 30:", OUT_COLS[:30])

export_df = df.copy()
export_df["date"] = pd.to_datetime(export_df["date"]).dt.strftime("%Y-%m-%d")
export_df[OUT_COLS].to_csv(OUT_PATH, index=False)

print("===================================")
print("✅ Model output saved:")
print(OUT_PATH.resolve())
print("Rows:", len(df))
print("===================================")


Z_EXPORT: 15 e.g. ['daily_event_cnt_z', 'Screen_z', 'Notif_z', 'UserAct_z', 'night_ratio_z', 'hour_entropy_z', 'gap_max_z', 'gap_p95_z', 'gap_cnt_2h_z', 'gap_cnt_6h_z']
Z_EARLY_EXPORT: 12 e.g. ['Screen_z_early', 'Notif_z_early', 'UserAct_z_early', 'daily_event_cnt_z_early', 'night_ratio_z_early', 'hour_entropy_z_early', 'gap_max_z_early', 'gap_cnt_2h_z_early', 'gap_long_ratio_z_early', 'overnight_gap_z_early']
Export columns: 44
First 30: ['uuid', 'date', 'day_idx', 'cold_stage', 'early_ready', 'early_risk', 'final_risk', 'quality_state', 'context_mode', 'baseline_ready', 'risk_score', 'risk_pre', 'risk_adj', 'z_score', 'a_score', 'z_trim_mean_abs', 'z_max_abs', 'anomaly_flag', 'anomaly_score', 'partial_flag', 'travel_flag', 'tz_flag', 'bad_history_flag', 'stable_hold_flag', 'drift_flag', 'drift_top_feature', 'daily_event_cnt_z', 'Screen_z', 'Notif_z', 'UserAct_z']
✅ Model output saved:
C:\Users\DS3\Desktop\amz\artifacts\model\model_out_v3.csv
Rows: 9057


# 8. 모델 상태 검사

In [ ]:
# =========================
# 8. Model Quality Report (No-Label)
# =========================
import numpy as np
import pandas as pd

def model_quality_report(df: pd.DataFrame,
                         risk_col="risk_score",
                         final_risk_col="final_risk",
                         mask_col="risk_mask",
                         quality_col="quality_state",
                         context_col="context_mode",
                         z_cols=None,
                         a_col="a_score",
                         t_col="t_score",
                         top_threshold=0.85,
                         severe_threshold=0.95,
                         z_ref_med=None): 

    d = df.copy()

    # -------------------------
    # 0) 기본 컬럼/마스크
    # -------------------------
    if (mask_col is not None) and (mask_col in d.columns):
        valid_mask = d[mask_col].fillna(False).astype(bool)
    else:
        valid_mask = d[risk_col].notna()

    # z cols 자동 수집
    if z_cols is None:
        z_cols = [c for c in d.columns if c.endswith("_z") and (not c.endswith("_z_early"))]

    z_cols = [c for c in z_cols if c in d.columns]

    # 숫자형 보정
    for c in [risk_col, final_risk_col, a_col, "z_score"]:
        if c in d.columns:
            d[c] = pd.to_numeric(d[c], errors="coerce")

    # -------------------------
    # 1) Coverage / Gate 분포
    # -------------------------
    out = {}
    out["n_rows"] = int(len(d))
    out["n_users"] = int(d["uuid"].nunique()) if "uuid" in d.columns else np.nan

    out["risk_valid_ratio"] = float(d[risk_col].notna().mean()) if risk_col in d.columns else np.nan
    out["final_risk_valid_ratio"] = float(d[final_risk_col].notna().mean()) if final_risk_col in d.columns else np.nan
    out["mask_ratio"] = float(valid_mask.mean())

    # gate 분포
    if quality_col in d.columns:
        out["quality_dist"] = d[quality_col].value_counts(normalize=True).round(3).to_dict()
    if context_col in d.columns:
        out["context_dist"] = d[context_col].value_counts(normalize=True).round(3).to_dict()

    # -------------------------
    # 2) Risk 분포 + “희귀성(Alarm rate)”
    # 운영에서 중요한 건 "위험이 너무 자주 뜨냐/안 뜨냐"
    # -------------------------
    dd = d.loc[valid_mask & d[risk_col].notna(), ["uuid","date",risk_col]].copy()
    if dd.empty:
        print("⚠️ no valid rows for report (check mask/risk).")
        return out

    out["risk_desc"] = dd[risk_col].describe(percentiles=[.5,.75,.9,.95,.99]).round(3).to_dict()
    out["alarm_rate(>=top_thr)"] = float((dd[risk_col] >= top_threshold).mean())
    out["severe_rate(>=sev_thr)"] = float((dd[risk_col] >= severe_threshold).mean())

    # 유저별 알람 빈도(너무 특정 유저에 몰리면 편향 가능)
    per_user_alarm = dd.groupby("uuid")[risk_col].apply(lambda s: float((s >= top_threshold).mean()))
    out["alarm_rate_per_user_desc"] = per_user_alarm.describe(percentiles=[.5,.9,.95]).round(3).to_dict()

    # -------------------------
    # 3) Temporal consistency (연속성)
    # 위험은 보통 “연속된 구간”으로 나타나는게 자연스러움
    # 반대로 하루 튀고 바로 정상 -> 노이즈 가능
    # -------------------------
    dd2 = d.loc[valid_mask & d[risk_col].notna(), ["uuid","date",risk_col]].copy()
    dd2["date"] = pd.to_datetime(dd2["date"], errors="coerce")
    dd2 = dd2.sort_values(["uuid","date"])

    is_alarm = (dd2[risk_col] >= top_threshold).astype(int)
    # 연속 알람(run length) 계산
    runs = []
    for uid, g in dd2.groupby("uuid"):
        x = (g[risk_col] >= top_threshold).astype(int).values
        if len(x) == 0:
            continue
        cur = 0
        for v in x:
            if v == 1:
                cur += 1
            else:
                if cur > 0:
                    runs.append(cur)
                cur = 0
        if cur > 0:
            runs.append(cur)

    if len(runs):
        runs = pd.Series(runs)
        out["alarm_runlen_desc"] = runs.describe(percentiles=[.5,.9,.95,.99]).round(3).to_dict()
    else:
        out["alarm_runlen_desc"] = {"count": 0}

    # -------------------------
    # 4) Component sanity (z/a/t 밸런스)
    # 특정 컴포넌트만 지배하면 위험점수가 편향됨
    # -------------------------
    comp_cols = [c for c in ["z_score", a_col] if c in d.columns]
    comp = d.loc[valid_mask, comp_cols].copy()
    comp = comp.apply(pd.to_numeric, errors="coerce")
    out["components_desc"] = comp.describe(percentiles=[.5,.9,.95,.99]).round(3).to_dict()

    # risk와의 상관(너무 1에 가깝거나 0에 가깝면 설계 재검토)
    corr = {}
    for c in comp_cols:
        x = d.loc[valid_mask, [risk_col, c]].dropna()
        corr[c] = float(x[risk_col].corr(x[c])) if len(x) > 30 else np.nan
    out["risk_component_corr"] = {k: (None if pd.isna(v) else round(v,3)) for k,v in corr.items()}

       # -------------------------
    # 5) Explainability proxy: 상위 위험일에서 “Top-Z 피처”
    # -------------------------
    if len(z_cols) > 0:
        zz = d.loc[valid_mask & d[risk_col].notna(), ["uuid","date",risk_col] + z_cols].copy()
        zz[z_cols] = zz[z_cols].apply(pd.to_numeric, errors="coerce")

        topk = zz.sort_values(risk_col, ascending=False).head(200)

        # ✅ ref_med: "정상/학습 풀" 기준으로 고정 (없으면 valid 전체로 fallback)
        if z_ref_med is None:
            z_ref_med = d.loc[valid_mask, z_cols].abs().median()

        z_ref_med = z_ref_med.replace(0, np.nan).fillna(1.0)

        abs_norm = topk[z_cols].abs().div(z_ref_med, axis=1)
        abs_norm2 = abs_norm.dropna(how="all")
        if len(abs_norm2) == 0:
            out["topZ_feature_freq(top200)"] = {}
        else:
            top1 = abs_norm2.idxmax(axis=1).value_counts().head(15)
            out["topZ_feature_freq(top200)"] = top1.to_dict()

    # -------------------------
    # 6) 출력
    # -------------------------
    print("\n==============================")
    print("📌 MODEL QUALITY REPORT (no-label)")
    print("==============================")
    print(f"rows={out['n_rows']}, users={out['n_users']}")
    print(f"mask_ratio={out['mask_ratio']:.3f}, risk_valid={out['risk_valid_ratio']:.3f}, final_risk_valid={out['final_risk_valid_ratio']:.3f}")
    print("\n[Gate dist]")
    if "quality_dist" in out: print("quality:", out["quality_dist"])
    if "context_dist" in out: print("context:", out["context_dist"])

    print("\n[Risk distribution]")
    print(pd.Series(out["risk_desc"]))

    print(f"\n[Alarm rate]  risk>={top_threshold}: {out['alarm_rate(>=top_thr)']:.3f} | risk>={severe_threshold}: {out['severe_rate(>=sev_thr)']:.3f}")
    print("\n[Per-user alarm rate desc]")
    print(pd.Series(out["alarm_rate_per_user_desc"]))

    print("\n[Alarm run-length desc] (연속 알람 길이)")
    print(pd.Series(out["alarm_runlen_desc"]))

    print("\n[Components desc]")
    # components_desc는 dict-of-dict 형태라 보기 좋게 요약
    for c in comp_cols:
        print(f" - {c}:",
              {k: out["components_desc"][c].get(k, None) for k in ["mean","50%","90%","95%","99%","max"] if k in out["components_desc"][c]})

    print("\n[Risk vs component correlation]")
    print(out["risk_component_corr"])

    if out["topZ_feature_freq(top200)"]:
        print("\n[Top-Z feature freq among Top200 risk days]")
        print(out["topZ_feature_freq(top200)"])

    print("==============================\n")
    return out

# -------------------------
# [PATCH] report용 z_cols / z_ref_med 안전 정의
# -------------------------
z_cols_report = [c for c in df.columns if c.endswith("_z") and (not c.endswith("_z_early"))]

# ref_med는 "정상+baseline_ready" 풀에서 고정 (없으면 valid 전체로 fallback)
pool_mask = (df["quality_state"] == "GOOD") & df["baseline_ready"].fillna(False)
if pool_mask.any() and len(z_cols_report) > 0:
    z_ref_med = df.loc[pool_mask, z_cols_report].abs().median()
else:
    z_ref_med = df.loc[df["final_risk"].notna(), z_cols_report].abs().median() if len(z_cols_report) > 0 else None

# ✅ 퍼센타일 기반 임계값 (스케일 천장 문제 무시하고 항상 알람이 생기게)
# ✅ final_risk 기준으로 임계값 계산 (콜드스타트 포함)
valid_alarm_pool = df.loc[df["risk_score"].notna(), "risk_score"]
if len(valid_alarm_pool) == 0:
    raise ValueError("risk_score is all NaN. Check risk_mask / pipeline.")

TOP_THR = float(valid_alarm_pool.quantile(0.99))
SEV_THR = float(valid_alarm_pool.quantile(0.998))

print("AUTO thresholds (risk_score):", {"TOP(0.99)": round(TOP_THR, 3), "SEV(0.998)": round(SEV_THR, 3)})

# ✅ report도 risk_score로 평가 (final_risk는 참고용으로만)
report8 = model_quality_report(
    df,
    risk_col="risk_score",
    final_risk_col="final_risk",
    mask_col=None,
    top_threshold=TOP_THR,
    severe_threshold=SEV_THR,
    z_cols=z_cols_report,
    z_ref_med=z_ref_med,
)

AUTO thresholds (risk_score): {'TOP(0.99)': 0.663, 'SEV(0.998)': 0.769}

📌 MODEL QUALITY REPORT (no-label)
rows=9057, users=342
mask_ratio=0.665, risk_valid=0.665, final_risk_valid=0.812

[Gate dist]
quality: {'GOOD': 0.961, 'LOW_CONF': 0.039}
context: {'NORMAL': 0.991, 'PARTIAL': 0.009}

[Risk distribution]
count    6024.000
mean        0.335
std         0.093
min         0.058
50%         0.321
75%         0.381
90%         0.449
95%         0.498
99%         0.663
max         0.838
dtype: float64

[Alarm rate]  risk>=0.6627854589604805: 0.010 | risk>=0.7694093657571148: 0.002

[Per-user alarm rate desc]
count    336.000
mean       0.011
std        0.043
min        0.000
50%        0.000
90%        0.043
95%        0.083
max        0.500
dtype: float64

[Alarm run-length desc] (연속 알람 길이)
count    37.000
mean      1.649
std       0.588
min       1.000
50%       2.000
90%       2.000
95%       2.200
99%       3.000
max       3.000
dtype: float64

[Components desc]
 - z_score: {'mean': 